In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt

print('Version', torch.__version__) 
print('CUDA enabled:', torch.cuda.is_available())

import os
# Running this should then print out:
# Version 1.7.0+cu101 (or something like this)
# CUDA enabled: True

Version 1.7.0+cu101
CUDA enabled: False


## Access the dataset 

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
#!ls /drive

Mounted at /content/drive/


In [ ]:
root = "/content/drive/MyDrive/AskaData/AskaData1/aska3bN/cell_small/"

## Prepare the data

Example of cell image

In [ ]:
im = plt.imread('Cell0000526_frame_2.png')
plt.imshow(im)

FileNotFoundError: ignored

In [ ]:
im.shape

(11, 31)

**Inputs into convolutional LSTM are shaped:**

**tensor of size B, T, C, H, W or T, B, C, H, W**

B = batch

T = timestep

C = channels

H = heights

W = width 

We are currently thinking we do not have to pad the images. 

In [ ]:
import imageio

In [ ]:
#Iterates over all of the files, but does not retain order

#Want to imread, turn into tensor, stack each image in the correct order of the tower
#Then want to stack each tower together
tower1 = torch.zeros(1,8,11,54)
data = []

for subdir, dirs, files in os.walk(root):
  tower = [None] * 8
  for file in files:
    #print(os.path.join(subdir, file))
    im = torch.LongTensor(imageio.imread(os.path.join(subdir, file)))

    #Padding
    p = (54 - im.shape[1])//2
    r = (54 - im.shape[1])%2
    m = nn.ConstantPad1d(padding=(p + r, p), value=0)
    im = m(im)
    
    try:
      i = int(os.path.join(subdir, file)[-5])
      tower[i - 1] = im
    except:
      pass

  if tower[0] != None:
    #Convert list (tower) of cells into tensor
    tower = torch.stack(tower, dim=0)
    data.append(tower)

Defining the dataloaders 

In [ ]:
n = len(data)
trainLength = (n//5)*4
testLength = n - trainLength

trainset, testset = torch.utils.data.random_split(data, [trainLength, testLength], 
                                                  generator=torch.Generator().manual_seed(42))
'''
d = trainLength
trainLength = (d//5)*4
valLength = d - trainLength
'''
# Lengths for the cell_small 
trainLength = 2
valLength = 2

trainset, valset = torch.utils.data.random_split(trainset, [trainLength, valLength], 
                                                  generator=torch.Generator().manual_seed(42))

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, 
                                        shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False)

## Defining the network

### Convolutional LSTM 
Adapted from [ndrplz](https://raw.githubusercontent.com/ndrplz/ConvLSTM_pytorch/master/convlstm.py).

In [ ]:
class ConvLSTMCell(nn.Module):

    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        """
        Initialize ConvLSTM cell.

        Parameters
        ----------
        input_dim: int
            Number of channels of input tensor.
        hidden_dim: int
            Number of channels of hidden state.
        kernel_size: (int, int)
            Size of the convolutional kernel.
        bias: bool
            Whether or not to add the bias.
        """

        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis

        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))


class ConvLSTM(nn.Module):

    """

    Parameters:
        input_dim: Number of channels in input
        hidden_dim: Number of hidden channels
        kernel_size: Size of kernel in convolutions
        num_layers: Number of LSTM layers stacked on each other
        batch_first: Whether or not dimension 0 is the batch or not
        bias: Bias or no bias in Convolution
        return_all_layers: Return the list of computations for all layers
        Note: Will do same padding.

    Input:
        A tensor of size B, T, C, H, W or T, B, C, H, W
    Output:
        A tuple of two lists of length num_layers (or length 1 if return_all_layers is False).
            0 - layer_output_list is the list of lists of length T of each output
            1 - last_state_list is the list of last states
                    each element of the list is a tuple (h, c) for hidden state and memory
    Example:
        >> x = torch.rand((32, 10, 64, 128, 128))
        >> convlstm = ConvLSTM(64, 16, 3, 1, True, True, False)
        >> _, last_states = convlstm(x)
        >> h = last_states[0][0]  # 0 for layer index, 0 for h index
    """

    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=False, bias=True, return_all_layers=False):
        super(ConvLSTM, self).__init__()

        self._check_kernel_size_consistency(kernel_size)

        # Make sure that both `kernel_size` and `hidden_dim` are lists having len == num_layers
        kernel_size = self._extend_for_multilayer(kernel_size, num_layers)
        hidden_dim = self._extend_for_multilayer(hidden_dim, num_layers)
        if not len(kernel_size) == len(hidden_dim) == num_layers:
            raise ValueError('Inconsistent list length.')

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers

        cell_list = []
        for i in range(0, self.num_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1]

            cell_list.append(ConvLSTMCell(input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))

        self.cell_list = nn.ModuleList(cell_list)

    def forward(self, input_tensor, hidden_state=None):
        """

        Parameters
        ----------
        input_tensor: todo
            5-D Tensor either of shape (t, b, c, h, w) or (b, t, c, h, w)
        hidden_state: todo
            None. todo implement stateful

        Returns
        -------
        last_state_list, layer_output
        """
        if not self.batch_first:
            # (t, b, c, h, w) -> (b, t, c, h, w)
            input_tensor = input_tensor.permute(1, 0, 2, 3, 4)

        b, _, _, h, w = input_tensor.size()

        # Implement stateful ConvLSTM
        if hidden_state is not None:
            raise NotImplementedError()
        else:
            # Since the init is done in forward. Can send image size here
            hidden_state = self._init_hidden(batch_size=b,
                                             image_size=(h, w))

        layer_output_list = []
        last_state_list = []

        seq_len = input_tensor.size(1)
        cur_layer_input = input_tensor

        for layer_idx in range(self.num_layers):

            h, c = hidden_state[layer_idx]
            output_inner = []
            for t in range(seq_len):
                h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, t, :, :, :],
                                                 cur_state=[h, c])
                output_inner.append(h)

            layer_output = torch.stack(output_inner, dim=1)
            cur_layer_input = layer_output

            layer_output_list.append(layer_output)
            last_state_list.append([h, c])

        if not self.return_all_layers:
            layer_output_list = layer_output_list[-1:]
            last_state_list = last_state_list[-1:]

        return layer_output_list, last_state_list

    def _init_hidden(self, batch_size, image_size):
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size, image_size))
        return init_states

    @staticmethod
    def _check_kernel_size_consistency(kernel_size):
        if not (isinstance(kernel_size, tuple) or
                (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))):
            raise ValueError('`kernel_size` must be tuple or list of tuples')

    @staticmethod
    def _extend_for_multilayer(param, num_layers):
        if not isinstance(param, list):
            param = [param] * num_layers
        return param

### ConvLSTMNet
Simple model with single convolution LSTM layer. Using this model as a baseline before we add in more convolutional layers and residual connections. 

In [ ]:
class ConvLSTMNet(nn.Module):
  def __init__(self, inputChannels, featureSize):
    super(ConvLSTMNet, self).__init__()

    self.c = inputChannels
    #self.p = numProteins
    self.featureSize = featureSize

    #ConvLSTM uses convolutional operations rather than regular matrix operations
    self.convLSTM = ConvLSTM(input_dim=1, hidden_dim=self.featureSize, kernel_size=[(3,3)], num_layers=1)
    #self.classify = nn.Linear(in_features=self.featureSize, out_features=self.p)


  def forward(x, hidden=None):
    """
    Takes in a batch of data (N, T, 12, 56). Data are sequences of images. The batch is a tensor
    with dimensions N: batch size, 56: width of image, 12: height of image, T: number of timesteps.
    Returns a tensor of shape (N, 1, 12, 56) which is the predicted next frame in the sequence.
    Also returns an encoded label indicating which protein type the sequence contains. 
        
    :param batch: Tensor of shape (N, T, 12, 56) Batch of time series frames. Dimensions: N 
    (number of samples) by 56 (width of image) by 12 (height of image) by T (number of timesteps). 

    HAVE NOT IMPLEMENTED THIS YET(
    :return prediction: Tensor of shape (N, 1, 12, 56). Dimensions: N (number of samples) by 
    56 (width of image) by 12 (height of image) by 1 (number of timesteps in the future). 
    )

    :return label: Tensor of shape (N, p). Dimensions: N (number of samples) by p (number of 
    protein types).
    """

    x, hidden = self.convLSTM(x, hidden)

    return x, hidden

  def inference(self, x, hidden=None, temperature=1):
    x = x.view(-1, 1)
    x, hidden_state = self.forward(x, hidden)
    x = x.view(1, -1)
    x = x / max(temperature, 1e-20)
    x = F.softmax(x, dim=1)
    return x, hidden

  def loss(self, prediction, label, reduction='mean'):
    loss_val = F.cross_entropy(prediction.view(-1, self.p), label.view(-1), reduction=reduction)
    return loss_val

  # Saves the current model
  def save_model(self, file_path, num_to_keep=1):
    pt_util.save(self, file_path, num_to_keep)

  # Saves the best model so far
  def save_best_model(self, accuracy, file_path, num_to_keep=1):
    if accuracy > self.best_accuracy:
      self.save_model(file_path, num_to_keep)
      self.best_accuracy = accuracy

  def load_model(self, file_path):
    pt_util.restore(self, file_path)

  def load_last_model(self, dir_path):
    return pt_util.restore_latest(self, dir_path)
  

### EcoliNet

Implements ConvLSTM as well as more convolutional layers and residual connections. 

In [ ]:
class EcoliNet(nn.Module):
  def __init__(self, inputChannels, numProteins, featureSize):
    super(EcoliNet, self).__init__()

    self.c = inputChannels
    self.p = numProteins
    self.featureSize = featureSize

    #Conv3D takes in inputs of (N,C,D,H,W)
    #   N: batch size
    #   C: number of channels
    #   D: number of frames
    #   H: height of image
    #   W: width of image 

    self.conv1 = nn.Conv3d(in_channels=self.c, out_channels=3, kernel_size=3, stride=1, padding=1)
    self.conv2 = nn.Conv3d(in_channels=3, out_channels=9, kernel_size=3, stride=1, padding=1)
    self.conv3 = nn.Conv3d(in_channels=9, out_channels=27, kernel_size=3, stride=1, padding=1)

    self.bn1 = nn.BatchNorm3d(num_features=3)
    self.bn2 = nn.BatchNorm3d(num_features=9)
    self.bn3 = nn.BatchNorm3d(num_features=27)

    #ConvLSTM uses convolutional operations rather than regular matrix operations
    self.convLSTM = ConvLSTM(input_dim=27, hidden_dim=self.featureSize, kernel_size=3)

    self.classify = nn.Linear(in_features=self.featureSize, out_features=self.p)


  def forward(batch, hidden=None):
    """
    Takes in a batch of data (N, T, 12, 56). Data are sequences of images. The batch is a tensor
    with dimensions N: batch size, 56: width of image, 12: height of image, T: number of timesteps.
    Returns a tensor of shape (N, 1, 12, 56) which is the predicted next frame in the sequence.
    Also returns an encoded label indicating which protein type the sequence contains. 
        
    :param batch: Tensor of shape (N, T, 12, 56) Batch of time series frames. Dimensions: N 
    (number of samples) by 56 (width of image) by 12 (height of image) by T (number of timesteps). 

    :return predication: Tensor of shape (N, 1, 12, 56). Dimensions: N (number of samples) by 
    56 (width of image) by 12 (height of image) by 1 (number of timesteps in the future). 

    :return label: Tensor of shape (N, p). Dimensions: N (number of samples) by p (number of 
    protein types).
    """

    x = batch

    x = self.conv1(x)
    x = self.bn1(x)

    x = self.conv2(x)
    x = self.bn2(x)

    x = self.conv3(x)
    x = self.bn3(x)

    x, hidden = self.convLSTM(x, hidden)

    x = nn.classify(hidden)
    #x = nn.classify(x[:,-1,:])

    return 

  def inference(self):

  def loss(self):

  # Saves the current model
  def save_model(self, file_path, num_to_keep=1):
    pt_util.save(self, file_path, num_to_keep)

  # Saves the best model so far
  def save_best_model(self, accuracy, file_path, num_to_keep=1):
    if accuracy > self.best_accuracy:
      self.save_model(file_path, num_to_keep)
      self.best_accuracy = accuracy

  def load_model(self, file_path):
    pt_util.restore(self, file_path)

  def load_last_model(self, dir_path):
    return pt_util.restore_latest(self, dir_path)

## Training

In [ ]:
model = ConvLSTMNet(inputChannels=1, featureSize=8)

In [ ]:
import tqdm
def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)


def train(model, device, optimizer, trainloader, valloader, lr, epoch, log_interval):
    model.train()
    losses = []
    valLosses = []
    hidden = None

    for i, data in enumerate(trainloader, 0):
        #Inputs are first 7 cells in a tower, outputs are last 7 cells in a tower 
        inputs = torch.stack([x[0:7,:,:] for x in data])
        outputs = torch.stack([x[1:8,:,:] for x in data])
        inputs = inputs.to(device)
        outputs = outputs.to(device)

        # Separates the hidden state across batches. 
        # Otherwise the backward would try to go all the way to the beginning every time.
        if hidden is not None:
            hidden = repackage_hidden(hidden)
        optimizer.zero_grad()
        preds, hidden = model(inputs)
        #pred = output.max(-1)[1]
        loss = model.loss(preds, outputs)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        if i % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(data), len(trainloader.dataset),
                100. * i / len(trainloader), loss.item()))    

    #Validation       
    for i, data in enumerate(valloader, 0):
        inputs = torch.stack([x[0:7,:,:] for x in data])
        outputs = torch.stack([x[1:8,:,:] for x in data])
        inputs = inputs.to(device)
        outputs = outputs.to(device)

        if hidden is not None:
          hidden = repackage_hidden(hidden)

        preds, hidden = model(inputs)
        #pred = output.max(-1)[1]
        loss = model.loss(preds, outputs)
        valLosses.append(loss.item())
        if i % log_interval == 0:
              print('Train Epoch: {} [{}/{} ({:.0f}%)]\tVal Loss: {:.6f}'.format(
                  epoch, i * len(data), len(valloader.dataset),
                  100. * i / len(valloader), loss.item()))
    return np.mean(losses), np.mean(valLosses)


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0 

    with torch.no_grad():
        hidden = None
        for i, data in enumerate(testloader, 0):
            inputs = torch.stack([x[0:7,:,:] for x in data])
            outputs = torch.stack([x[1:8,:,:] for x in data])
            inputs = inputs.to(device)
            outputs = outputs.to(device)

            preds, hidden = model(inputs, hidden)
            test_loss += model.loss(preds, outputs, reduction='mean').item()
            #pred = output.max(-1)[1]
            correct_mask = preds.eq(label.view_as(preds))
            num_correct = correct_mask.sum().item()
            correct += num_correct

    test_loss /= len(testloader)

    '''
    test_accuracy = 100. * correct / (len(testloader.dataset) * testloader.dataset.sequence_length)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset) * test_loader.dataset.sequence_length,
        100. * correct / (len(test_loader.dataset) * test_loader.dataset.sequence_length)))
        '''
        #, test_accuracy
    return test_loss

In [ ]:
def main(BATCH_SIZE = 256, FEATURE_SIZE = 64, TEST_BATCH_SIZE = 256, EPOCHS = 2, LEARNING_RATE = 0.002, WEIGHT_DECAY = 0.0005, EXP_VER = '1'):
  
    PRINT_INTERVAL = 10
    USE_CUDA = True
    LOG_PATH = root + EXP_VER + 'logs/log.pkl'

    data_train = trainset
    data_val = valset
    data_test = testset

    use_cuda = USE_CUDA and torch.cuda.is_available()

    device = torch.device("cuda" if use_cuda else "cpu")
    print('Using device', device)
    import multiprocessing
    num_workers = multiprocessing.cpu_count()
    print('num workers:', num_workers)

    kwargs = {'num_workers': num_workers,
              'pin_memory': True} if use_cuda else {}


    trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                              shuffle=True)
    valloader = torch.utils.data.DataLoader(valset, batch_size=BATCH_SIZE, 
                                            shuffle=True)
    testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                             shuffle=False)
    '''
    train_loader = torch.utils.data.DataLoader(data_train, batch_size=BATCH_SIZE,
                                              shuffle=False, **kwargs)
    val_loader = torch.utils.data.DataLoader(data_val, batch_size=BATCH_SIZE,
                                            shuffle=False, **kwargs)
    test_loader = torch.utils.data.DataLoader(data_test, batch_size=TEST_BATCH_SIZE,
                                              shuffle=False, **kwargs)
    '''
    #May need to change some of these parameters? 
    model = ConvLSTMNet(inputChannels=1, featureSize=FEATURE_SIZE).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

    start_epoch = 0

    #train_losses, val_losses, test_losses, test_accuracies = pt_util.read_log(LOG_PATH, ([], [], [], []))
    test_loss, test_accuracy = test(model, device, testloader)

    test_losses.append((start_epoch, test_loss))
    test_accuracies.append((start_epoch, test_accuracy))

    try:
      for epoch in range(start_epoch, EPOCHS + 1):
        lr = LEARNING_RATE * np.power(0.25, (int(epoch / 6)))
        train_loss, val_loss = train(model, device, optimizer, trainloader, lr, epoch, PRINT_INTERVAL)
        test_loss, test_accuracy = test(model, device, testloader)

        train_losses.append((epoch, train_loss))
        val_losses.append((epoch, val_loss))
        print(train_losses)
        print(val_losses)
        test_losses.append((epoch, test_loss))
        test_accuracies.append((epoch, test_accuracy))
        #pt_util.write_log(LOG_PATH, (train_losses, val_losses, test_losses, test_accuracies))
        model.save_best_model(test_accuracy, DATA_PATH + 'checkpoints/%03d.pt' % epoch)

    except KeyboardInterrupt as ke:
      print('Interrupted')
    except:
      import traceback
      traceback.print_exc()
    finally:
      print('Saving final model')
      model.save_model(DATA_PATH + 'checkpoints/%03d.pt' % epoch, 0)

      #ep, val = zip(*train_losses)
      #pt_util.plot(ep, val, 'Train loss', 'Epoch', 'Error')

      #ep, val = zip(*val_losses)
      #pt_util.plot(ep, val, 'Validation loss', 'Epoch', 'Error')

      #ep, val = zip(*test_losses)
      #pt_util.plot(ep, val, 'Test loss', 'Epoch', 'Error')

      #ep, val = zip(*test_accuracies)
      #pt_util.plot(ep, val, 'Test accuracy', 'Epoch', 'Percent correct')
      return model, device


In [ ]:
model, device = main(BATCH_SIZE = 1, FEATURE_SIZE = 3, TEST_BATCH_SIZE = 1, EPOCHS = 2, LEARNING_RATE = 0.002, WEIGHT_DECAY = 0.0005, EXP_VER = '1')

Using device cpu
num workers: 2


TypeError: ignored